In [1]:
import getpass
import os

In [2]:
# install langchain-core and langgraph%
%pip install langchain-core langgraph>0.2.27
%pip install python-dotenv
%pip install langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from collections.abc import Sequence
from typing import Annotated

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

/Users/gcyang/Documents/Barking Dog/python-course/chatbot/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
load_dotenv()
api_key_l = os.getenv("LANG_SMITH_API_KEY")
type(api_key_l)

str

In [5]:
if not os.environ.get("LANG_SMITH_KEY"):
    os.environ["LANG_SMITH_KEY"] = getpass.getpass(api_key_l)

In [6]:
# connect langsmith
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = api_key_l

In [7]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [8]:
# set the GOOGLE_KEY environment
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(api_key)

In [9]:
pip install -U langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip show langchain-google-genai

Name: langchain-google-genai
Version: 2.1.6
Summary: An integration package connecting Google's genai package and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MIT
Location: /Users/gcyang/Documents/Barking Dog/python-course/chatbot/.venv/lib/python3.9/site-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [11]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [46]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


# Define a new graph
workflow = StateGraph(state_schema=State)

In [47]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [48]:
trimmer = trim_messages(
    max_tokens=500,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [49]:
class ModelOutput(TypedDict):
    messages: AIMessage


# Define to call the model
def call_model(state: State) -> ModelOutput:
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": response}

In [50]:
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [51]:
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [52]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Dog. Write me a self introduction with 100 words."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, _ in app.stream(
    {"messages": input_messages, "language": language},
    config=config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")  # noqa: T201

Wo|of wo|of! Hey there, humans! I'm Dog, and I'm super| excited to make your acquaintance! I'm a devoted companion with a heart full of| love and a tail that's always wagging. My favorite things include chasing tennis balls, getting ear scratches, and snuggling up for a good nap. I'|m a master of the puppy-dog eyes and can usually convince anyone to share their snacks. I promise to be a loyal and playful friend, always ready to brighten your day| with a happy bark and a wet nose. Let's go for a walk and become best buds! Arf!
|

In [56]:
query = "Who am I?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config=config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Based on our conversation, you are Dog! You've introduced yourself as Dog in all your prompts. Woof!
